# SHapley Additive exPlanations (SHAP)

## Library Imports 

In [ ]:
import os
import pickle

import pandas as pd
import numpy as np
import shap
from matplotlib import pyplot as plt

In [ ]:
RAND_STATE = 0

## Importing the Test Sets

In [ ]:
dataset_folder = f"{os.path.abspath(os.path.join(os.getcwd(), os.pardir))}/datasets"

In [ ]:
X_test = pd.read_csv(os.path.join(dataset_folder, "obesity_X_test.csv"), index_col=0)

As with the rest of the stages, we make a variant of the sets without the height and weight columns for comparison:

In [ ]:
X_test_no_hw = X_test.drop(["Height", "Weight"], axis=1)

## Importing the Random Forest Classifiers
We import the random forest classifiers that we trained previously. SHAP values will be calculated for these models to explain their predictions.

In [ ]:
def import_model(filename):
    file_path = f"{os.path.abspath(os.path.join(os.getcwd(), os.pardir))}/models/{filename}"
    with open(file_path, 'rb') as file: 
        model = pickle.load(file)
    print(f"Model imported from {file_path}")
    return model

In [ ]:
rand_forest = import_model("rand_forest.pkl")

In [ ]:
rand_forest_no_hw = import_model("rand_forest_no_hw.pkl")

## Importing Encoders
We import the label encoder for the target feature so that we can encode the original values for indexing purposes.

In [ ]:
def import_encoder(filename):
    file_path = f"{os.path.abspath(os.path.join(os.getcwd(), os.pardir))}/encoders/{filename}"
    with open(file_path, 'rb') as file: 
        encoder = pickle.load(file)
    print(f"Encoder imported from {file_path}")
    return encoder

In [ ]:
target_le = import_encoder("target_le.pkl")

In [ ]:
target_class_label_d = {cls: idx for idx, cls in enumerate(target_le.classes_)}
target_class_label_d

We also import the one-hot encoder and scaler, which are needed when processing the SHAP values:

In [ ]:
nominal_ohe = import_encoder("nominal_ohe.pkl")

In [ ]:
scaler = import_encoder("scaler.pkl")

## Explanations

### Calculating the SHAP Values

First, we create the explainers and calculate the SHAP values:

In [ ]:
explainer = shap.TreeExplainer(rand_forest)
shap_values = explainer(X_test)

In [ ]:
explainer_no_hw = shap.TreeExplainer(rand_forest_no_hw)
shap_values_no_hw = explainer_no_hw(X_test_no_hw)

### Preprocessing

#### Undoing the One-Hot Encoding
However, because our nominal features are one-hot encoded, their SHAP values have been "separated". We need to group these one-hot encoded features back together. For reference, here is a list of all column after one-hot encoding:

In [ ]:
X_test.columns

##### Summing Shape Values
We first need to calculate the SHAP values for each nominal feature, which is done by taking the sum of the SHAP values for each feature's corresponding set of one-hot encoded features. We begin by determining the number of output one-hot encoded features for each nominal feature:

In [ ]:
n_ohe_feats: dict[str, int] = {
    feat_name: (len(categories) if drop_idx is None else len(categories) - 1)
    for feat_name, categories, drop_idx
    in zip(nominal_ohe.feature_names_in_, nominal_ohe.categories_, nominal_ohe.drop_idx_)
}
n_ohe_feats

Our one-hot encoded dataset has the one-hot encoded features after all numerical features.
Hence, the number of numerical features will tell us the start index of the first one-hot encoded feature:

In [ ]:
numerical_feature_count = 8
numerical_feature_count_no_hw = 6

We will now define a function to perform the summation for us.

To calculate the sum, we first group the SHAP value columns by their original nominal features.
Then, we sum the SHAP values for each group column-wise.
After that, we simply concatenate back the SHAP values for the numerical features.

In [ ]:
def undo_shap_values_ohe(svals, with_hw=True):
    num_feat_count = numerical_feature_count if with_hw else numerical_feature_count_no_hw
    # Split the SHAP values for the one-hot encoded features.
    # Each entry in split contains the SHAP values for the multiple one-hot encoded features of the original categorical feature.
    values_split = np.split(svals.values[:, num_feat_count:, :], np.cumsum(list(n_ohe_feats.values())[:-1]), axis=1)
    
    # Sum the SHAP values for each group.
    values_summed = np.array([vals.sum(axis=1) for vals in values_split])
    
    # We need to swap the first two axes since the first axis should index the instances and the second axis should index the features.
    unohe_values = np.swapaxes(values_summed, 0, 1)
    
    # Finally, we concatenate back the SHAP values for the numerical features.
    new_values = np.concatenate((svals.values[:, :num_feat_count, :], unohe_values), axis=1)
    
    return new_values

We now apply the function to our two sets of SHAP values:

In [ ]:
new_values = undo_shap_values_ohe(shap_values)
new_values_no_hw = undo_shap_values_ohe(shap_values_no_hw, with_hw=False)

As a sanity check, we check the shape. The first axis's value should be the same as the number of instances, the second axis's value should be the number of features before one-hot encoding and the third axis's value should be `2` since we only have two different categories for the target feature.

In [ ]:
new_values.shape

In [ ]:
new_values_no_hw.shape

At last, we can replace the old SHAP values:

In [ ]:
shap_values.values = new_values
shap_values_no_hw.values = new_values_no_hw

##### Fixing the Data Values

Unfortunately, we are not done. We've replaced the SHAP values, but the data values are still one-hot encoded!
We will need to undo the one-hot encoding, which is, thankfully, fairly straightforward since we can reuse our one-hot encoder.

In [ ]:
def undo_shap_data_ohe(svals, with_hw=True):
    num_feature_count = numerical_feature_count if with_hw else numerical_feature_count_no_hw
    unohe_data = nominal_ohe.inverse_transform(svals.data[:, num_feature_count:])
    new_data = np.concatenate((svals.data[:, :num_feature_count], unohe_data), axis=1)
    return new_data

In [ ]:
new_data = undo_shap_data_ohe(shap_values)
new_data_no_hw = undo_shap_data_ohe(shap_values_no_hw, with_hw=False)

As before, we check the shape as a sanity check:

In [ ]:
new_data.shape

In [ ]:
new_data_no_hw.shape

Finally, we replace the data:

In [ ]:
shap_values.data = new_data

In [ ]:
shap_values_no_hw.data = new_data_no_hw

##### Fixing Feature Names
Lastly, we need to update the feature names since the old feature names includes the one-hot encoded features. This is straightforward:

In [ ]:
ohe_feat_names = [name if name != "family_history_with_overweight" else "Family History of Overweightness" for name in n_ohe_feats.keys()]

In [ ]:
shap_values.feature_names = X_test.columns[:numerical_feature_count].to_list() + ohe_feat_names
shap_values.feature_names

In [ ]:
shap_values_no_hw.feature_names = X_test_no_hw.columns[:numerical_feature_count_no_hw].to_list() + ohe_feat_names
shap_values_no_hw.feature_names

#### Undoing the Standardisation
To make it easier to interpret the graphs later on, we will undo the standardisation for SHAP values' `.data`. We define a function for performing this task:

In [ ]:
feature_indices = {name: idx for idx, name in enumerate(shap_values.feature_names) if name in scaler.feature_names_in_}
feature_indices_no_hw = {name: idx for idx, name in enumerate(shap_values_no_hw.feature_names) if name in scaler.feature_names_in_}

height_col = feature_indices["Height"]
weight_col = feature_indices["Weight"]

def undo_shap_data_scaling(svals, with_hw=True):
    feat_indices = list(feature_indices.values()) if with_hw else list(feature_indices_no_hw.values())
    numerical_data = svals.data[:, feat_indices]
    
    if not with_hw:
        # A little hacky — the scaler expects the height and weight columns to be there, but they aren't,
        # so we add fake height and weight columns temporarily, then remove them after scaling.
        fake_col_1 = min(height_col, weight_col)
        fake_col_2 = max(height_col, weight_col)
        
        copy_with_fake = np.zeros((numerical_data.shape[0], numerical_data.shape[1] + 2))
        copy_with_fake[:, :fake_col_1] = numerical_data[:, :fake_col_1]
        copy_with_fake[:, fake_col_1 + 1:fake_col_2] = numerical_data[:, fake_col_1:fake_col_2]
        copy_with_fake[:, fake_col_2 + 1:] = numerical_data[:, fake_col_2 - 1:]
        
        numerical_data = copy_with_fake.astype("float64")
        
    numerical_data = scaler.inverse_transform(numerical_data)

    # Drop the fake columns.
    if not with_hw:
        numerical_data = np.delete(numerical_data, [height_col, weight_col], axis=1)

    new_data = svals.data.copy()
    new_data[:, feat_indices] = numerical_data

    return new_data

Now, we undo the standardisation for both sets of SHAP values:

In [ ]:
shap_values.data = undo_shap_data_scaling(shap_values)
shap_values_no_hw.data = undo_shap_data_scaling(shap_values_no_hw, with_hw=False)

### Utility Functions

We define a function to help us plot a boxplot for a categorical feature against the SHAP values:

In [ ]:
# Adapted from: https://towardsdatascience.com/shap-for-categorical-features-7c63e6a554ea
def boxplot_categories(svals, feature: str, target_class: int, feature_display: str = None, transform_category=lambda x: x):
    values = svals[:, feature, target_class].values
    data = svals[:, feature, target_class].data
    categories = np.unique(data)
    
    groups = []
    for c in categories:
        relevant_values = values[data == c]
        groups.append(relevant_values)
    
    labels = [transform_category(category) for category in categories]
    
    plt.figure(figsize=(8, 5))
    plt.boxplot(groups, tick_labels=labels)
    plt.ylabel('SHAP Values', size=15)
    plt.xlabel(feature_display if feature_display is not None else feature, size=15);

### With Height and Weight
We can now plot graphs for the SHAP values. Since the SHAP values for the non-obese class are just the negation of those of the obese class,
it is sufficient to plot the values for only one of the classes. The following is a beeswarm diagram for the obese class:

In [ ]:
shap.plots.beeswarm(shap_values[:, :, target_class_label_d["Yes"]], max_display=X_test.shape[1])

Scatter plot for `FCVC`:

In [ ]:
shap.plots.scatter(shap_values[:, "FCVC", target_class_label_d["Yes"]])

Scatter plot for `FAF`:

In [ ]:
shap.plots.scatter(shap_values[:, "FAF", target_class_label_d["Yes"]])

Scatter plot for `Age`:

In [ ]:
shap.plots.scatter(shap_values[:, "Age", target_class_label_d["Yes"]])

A boxplot for the family history of overweightness feature:

In [ ]:
boxplot_categories(
    shap_values,
    "Family History of Overweightness",
    target_class_label_d["Yes"],
    transform_category=lambda c: c.title()
)

A box plot for `CAEC`:

In [ ]:
boxplot_categories(
    shap_values,
    "CAEC",
    target_class_label_d["Yes"],
    transform_category=lambda c: c.title()
)

A boxplot for `FAVC`:

In [ ]:
boxplot_categories(
    shap_values,
    "FAVC",
    target_class_label_d["Yes"],
    transform_category=lambda c: c.title()
)

### Without Height and Weight
Similarly, we plot graphs for the model trained without the height and weight.

In [ ]:
shap.plots.beeswarm(shap_values_no_hw[:, :, target_class_label_d["Yes"]], max_display=X_test_no_hw.shape[1])

Scatter plot for `FCVC`:

In [ ]:
shap.plots.scatter(shap_values_no_hw[:, "FCVC", target_class_label_d["Yes"]])

Scatter plot for `FAF`:

In [ ]:
shap.plots.scatter(shap_values_no_hw[:, "FAF", target_class_label_d["Yes"]])

Scatter plot for `Age`:

In [ ]:
shap.plots.scatter(shap_values_no_hw[:, "Age", target_class_label_d["Yes"]])

A boxplot for the `family_history_with_overweight` feature:

In [ ]:
boxplot_categories(
    shap_values_no_hw,
    "Family History of Overweightness",
    target_class_label_d["Yes"],
    transform_category=lambda c: c.title()
)

A box plot for `CAEC`:

In [ ]:
boxplot_categories(
    shap_values_no_hw,
    "CAEC",
    target_class_label_d["Yes"],
    transform_category=lambda c: c.title()
)

A boxplot for `FAVC`:

In [ ]:
boxplot_categories(
    shap_values_no_hw,
    "FAVC",
    target_class_label_d["Yes"],
    transform_category=lambda c: c.title()
)

A box plot for `MTRANS`:

In [ ]:
boxplot_categories(
    shap_values_no_hw,
    "MTRANS",
    target_class_label_d["Yes"],
    transform_category=lambda c: c.title()
)